In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
import numpy as np

In [3]:
from src.utils.DatabaseConnect import DatabaseConnect

# single machine setup (mongo is running here localy)
# "ip a" for ip address
MY_DATABASE_LINK = 'mongodb://172.26.234.237:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

DatabaseConnect.DB_LINK = MY_DATABASE_LINK

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['172.26.234.237:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [4]:
from src.utils.CacheAdapter import JSONAdapter, JSONMultiFileAdapter, EXP_END_OF_DATA
from src.utils.DatasetManager import ProjectsDatasetManager
from src.utils.validators import projectDataIsSufficient

In [5]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

In [6]:
import gensim
from gensim.models.doc2vec import TaggedDocument

In [7]:
CACHE_FILE_NAME = "cache__31-03-2025__(sufficient)_{index}.json"

In [8]:
# using adapter to load data from the cache files

class DataFeeder(ProjectsDatasetManager):
    def __init__(self, batchSize = 32):
        #super().__init__(batchSize, validator, adapter)
        self.batchSize = batchSize

    def feedFromCache(self, cacheAdapter):
        # generator function, that will use "JSONMultifileAdapter" to parse data from multiple cache file, collect projects into a list and feed to the model
        tempStorage = []
        #self.cacheAdapter = cacheAdapter

        i = 0
        while True:
            try:
                while len(tempStorage) >= self.batchSize:
                    yield tempStorage[:self.batchSize]
                    tempStorage = tempStorage[self.batchSize:]

                data = cacheAdapter.load(1) # load users one by one
                data = flatternData(data)
                tempStorage.extend(data)

                i += 1

            except EXP_END_OF_DATA:
                break

        yield tempStorage

def feedProjectsFromCache(manager, batchSize = 32):
    cacheFileName = CACHE_FILE_NAME
    tempStorage = []

    i = 0
    while True:
        try:
            while len(tempStorage) >= batchSize:
                yield tempStorage[:batchSize]
                tempStorage = tempStorage[batchSize:]

            manager.cacheAdapter.collectionName = cacheFileName.format(index = i)
            data = flatternData(manager.fromCache())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            # no data left
            break

    yield tempStorage

def feedOneByOne(manager):
    # will feed preprocessed projects data as TaggedDocument instances one by one
    cacheFileName = CACHE_FILE_NAME
    tempStorage = [] # stemporary storage for data, that was read from files

    i = 0
    while True:
        try:
            while len(tempStorage) >= 1:
                doc = tempStorage[0]
                yield TaggedDocument(words = doc["tokens"], tags = doc["tags"])
                tempStorage = tempStorage[1:]

            manager.cacheAdapter.collectionName = cacheFileName.format(index = i)
            data = flatternData(manager.fromCache())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            # no data left
            break



In [9]:
adapter = JSONAdapter()
ProjectsDatasetManager.usersCollection = usersCollection
ProjectsDatasetManager.projectsCollection = projectsCollection
manager = ProjectsDatasetManager(25, validate = projectDataIsSufficient, cacheAdapter = adapter)

In [10]:
"""
manager.fromDB()
textData = manager.getTextOnly()
for text in flatternData(textData):
    print(text)
    #print(flatternData(data))
    #print(text)
"""
def feedTextData(manager, batchSize = 1):
    tempStorage = []

    i = 0
    while True:
        try:
            while len(tempStorage) >= batchSize:
                if batchSize > 1:
                    yield tempStorage[:batchSize]
                else:
                    yield tempStorage[:batchSize][0] # if I'm requesting only one item per time (training one by one) -> just yield it
                tempStorage = tempStorage[batchSize:]

            manager.fromDB()
            data = flatternData(manager.getTextOnly())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            break

    yield tempStorage

i = 2
for textBatch in feedTextData(manager, 32):
    if i <= 0: break
    print(len(textBatch))
    for text in textBatch:
        print(text)
    i -= 1


Scanning user: github:betiol
Scanning user: github:DestinyJun
Scanning user: github:madf12
Scanning user: github:hectorqin
Scanning user: github:yaojunguang
Scanning user: github:romanofficial
Scanning user: github:mikedemarais
Scanning user: github:Redseb
Scanning user: github:anhquan291
Scanning user: github:arfanliaqat
Scanning user: github:zhazihong
Scanning user: github:xiaomao996688
Scanning user: github:FelipeMolinari
Scanning user: github:MohamadKh75
Scanning user: github:pilotocheg
Scanning user: github:TPXP
Scanning user: github:gattleung
Scanning user: github:872409
Scanning user: github:yusufbadurohman
Scanning user: github:EQuimper
Scanning user: github:neo355
Scanning user: github:13627491210
Scanning user: github:MikeChugunov
Scanning user: github:452MJ
Scanning user: github:mohamed-sultan
32
awesome-code-streamers List of code streamers from multiples plataforms like Twitch, Youtube, etc
smart-home-mqtt Experimental and WIP - Bridge between Google Smart Home and MQTT 
t

In [11]:

i = 2
for item in feedOneByOne(manager):
    if i == 0: break
    print(item)
    i -= 1

cache__31-03-2025__(sufficient)_0.json
TaggedDocument<['awesome', 'code', 'streamer', 'list', 'code', 'streamer', 'multiple', 'plataforms', 'like', 'twitch', 'youtube'], ['github:lucasfloriani/awesome-code-streamers', 'awesome-code-streamers', '', 'awesome', 'awesome-list', 'lists', 'resources']>
TaggedDocument<['smart', 'home', 'mqtt', 'experimental', 'wip', 'bridge', 'google', 'smart', 'home', 'mqtt'], ['github:alvarowolfx/smart-home-mqtt', 'smart-home-mqtt', 'JavaScript']>


In [16]:
class EXP_FEEDER_IS_NONE(Exception):
    def __init__(self):
        super().__init__("'Model.corpus' object must be an iterable structure!")

class EXP_MANAGER_IS_NONE(Exception):
    def __init__(self):
        super().__init__("'Model.manager' object must be a DatasetManager instance!")


class Model(gensim.models.doc2vec.Doc2Vec):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.corpus = None # corpus is an iterator(iterable function or class object), that will be run with "for" loop to extract batches of data
        self.manager = None
    
    def train(self):
        # will build vocabulary and train the model on corpus (corpus will be fed by corpus)

        if self.corpus == None: raise EXP_FEEDER_IS_NONE
        if not isinstance(self.manager, ProjectsDatasetManager) : raise EXP_MANAGER_IS_NONE

        self.build_vocab(self.corpus(manager))

        super().train(self.corpus(manager), total_examples=self.corpus_count, epochs=self.epochs)

    def assess(self):
        pass


In [13]:
"""
documentsCorpus = []

for project in manager.data:
    documentsCorpus.append(TaggedDocument(words=project["tokens"], tags=project["tags"]))

documentsCorpus
"""

'\ndocumentsCorpus = []\n\nfor project in manager.data:\n    documentsCorpus.append(TaggedDocument(words=project["tokens"], tags=project["tags"]))\n\ndocumentsCorpus\n'

In [18]:
# creating model

VECTOR_SIZE = 100
EPOCH_NUMBER = 6
WORD_MIN_AMOUNT = 3
WINDOW_SIZE = 7

model = Model(vector_size = VECTOR_SIZE, window = WINDOW_SIZE, min_count = WORD_MIN_AMOUNT, epochs = EPOCH_NUMBER)
model.corpus = feedOneByOne
model.manager = manager
model.train()
print(model.epochs)
#model.build_vocab(documentsCorpus)
#model.train(documentsCorpus, total_examples = model.corpus_count, epochs = model.epochs)

cache__31-03-2025__(sufficient)_0.json
cache__31-03-2025__(sufficient)_1.json
cache__31-03-2025__(sufficient)_2.json
cache__31-03-2025__(sufficient)_3.json
cache__31-03-2025__(sufficient)_4.json
cache__31-03-2025__(sufficient)_5.json
cache__31-03-2025__(sufficient)_6.json
cache__31-03-2025__(sufficient)_7.json
cache__31-03-2025__(sufficient)_8.json
cache__31-03-2025__(sufficient)_9.json
cache__31-03-2025__(sufficient)_10.json
cache__31-03-2025__(sufficient)_11.json
cache__31-03-2025__(sufficient)_12.json
cache__31-03-2025__(sufficient)_13.json
cache__31-03-2025__(sufficient)_14.json
cache__31-03-2025__(sufficient)_15.json
cache__31-03-2025__(sufficient)_16.json
cache__31-03-2025__(sufficient)_17.json
cache__31-03-2025__(sufficient)_18.json
cache__31-03-2025__(sufficient)_19.json
cache__31-03-2025__(sufficient)_20.json
cache__31-03-2025__(sufficient)_21.json
cache__31-03-2025__(sufficient)_22.json
cache__31-03-2025__(sufficient)_23.json
cache__31-03-2025__(sufficient)_24.json
cache__31-

TypeError: Using a generator as corpus_iterable can't support 6 passes. Try a re-iterable sequence.

In [ ]:
vector = model.infer_vector(['awesome', 'code', 'streamer', 'list', 'code', 'streamer', 'multiple', 'plataforms', 'like', 'twitch', 'youtube'])
vector